In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from transformers import AutoProcessor, AutoModelForImageClassification
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/fs/ess/PAS2136/Rayeed/BritishCarabids/British Carabids/BritishCarabids.csv"
brit_df = pd.read_csv(file_path)
brit_df.rename(columns={"CanonicalName": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = brit_df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum
1,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "microsoft/swinv2-base-patch4-window8-256"

processor = AutoProcessor.from_pretrained(model_name, use_fast=True)

model = AutoModelForImageClassification.from_pretrained(model_name).to(device).eval()


In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        features = outputs.logits
    
    return features.cpu().numpy()


In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"])])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)




  0%|          | 0/63077 [00:00<?, ?it/s]

In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

seed = 45

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

metrics = {}

predictions = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):

    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")

metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 12.27% | Prec: 21.22% | Rec: 12.27% | F1: 12.13% | Bal Acc: 16.38% | MCC: 0.1210
LogisticRegression        | Acc: 78.37% | Prec: 79.02% | Rec: 78.37% | F1: 78.45% | Bal Acc: 75.52% | MCC: 0.7825
NearestNeighbor           | Acc: 34.59% | Prec: 38.69% | Rec: 34.59% | F1: 34.66% | Bal Acc: 29.87% | MCC: 0.3424
MLP_Baseline              | Acc: 77.66% | Prec: 78.16% | Rec: 77.66% | F1: 77.63% | Bal Acc: 75.76% | MCC: 0.7754


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})

In [9]:
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
6930,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Harpalus rubripes,Anisodactylus nemorivagus,Harpalus rubripes,Harpalus rubripes,Harpalus rubripes
2549,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Bembidion fumigatum,Bembidion fumigatum,Bembidion fumigatum,Bembidion bualei,Bembidion fumigatum


In [10]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.122701,0.212208,0.122701,0.121265,0.163759,0.121004
LogisticRegression,LogisticRegression,0.783687,0.790173,0.783687,0.784544,0.755152,0.782535
NearestNeighbor,NearestNeighbor,0.34591,0.386933,0.34591,0.346604,0.298663,0.342401
MLP_Baseline,MLP_Baseline,0.776554,0.781639,0.776554,0.776347,0.75756,0.775385


In [11]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BritishCarabids/5.SWINv2-BritishCarabids-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BritishCarabids/5.SWINv2-BritishCarabids-species-metrics.csv", index=False)